# Age and Biomass

This script outputs the satellite-based rasters into the Google Earth Engine Cloud.

Inputs:




In [52]:
import ee
import geemap
from gee_0_utils import *
initialize()

config = ProjectConfig()
roi = config.roi
data_folder = config.data_folder
last_year = config.last_year

mapbiomas, lulc, fire = desired_lulc()


## Biomass - Export secondary

Biomass data is in hectares, but mapbiomas data is 30m resolution.

To deal with edge pixels, we aggregate biomass values to 30m by using the mean (so there is a buffer for land use pixels caught in between two biomass values).

## Remove isolated pixels - Export one_hectare_mask

In the map, there were isolated pixels, often around the edges of forest patches. These would likely be due to misclassification, or follow different behaviors due to edge effects.

To avoid this issue, a kernel is applied here to include only secondary forest patches that are mostly surrounded by other secondary forest pixels.

In [53]:
# Load the image collections
transition = ee.ImageCollection('projects/JRC/TMF/v1_2023/TransitionMap_Subtypes').mosaic().clip(roi)
annual_changes = ee.ImageCollection('projects/JRC/TMF/v1_2023/AnnualChanges').mosaic().clip(roi)

# Define regrowth and degraded conditions
regrowth = transition.gte(31).And(transition.lte(33))

# Initialize AgeRegrowth and AgeDegraded
tmf = ee.Image.constant(0)

# Calculate AgeRegrowth
for i in range(1990, last_year):
    year = 'Dec' + str(i)
    annual_changes_year = annual_changes.select(year)
    condition = annual_changes_year.eq(4).And(regrowth) # were regrowing then AND are regrowing now
    tmf = tmf.add(condition.eq(1))

tmf = tmf.selfMask().rename(f"tmf_{last_year}")


In [54]:
def export_secondary(age, name):

    # select the data points that match their classification as young or old by IPCC
    filter1 = ee.Image(f"{data_folder}/raw/00N_10N")
    filter2 = ee.Image(f"{data_folder}/raw/10N_20N")
    filter_ages = ee.ImageCollection([filter1, filter2]).mosaic()
    filter_young_secondary = filter_ages.eq(2)
    filter_old_secondary = filter_ages.eq(3)
    young_secondary = age.lte(20).updateMask(filter_young_secondary).unmask(0)
    old_secondary = age.gt(20).updateMask(filter_old_secondary).unmask(0)
    ipcc_mask = young_secondary.add(old_secondary)

    # ----------------  Select only patches of at least one hectare  ----------------
    # Identify all connected components (patches) in the image
    age_patches = age.connectedComponents(ee.Kernel.plus(1), 256)
    # Calculate the area of each patch
    patchAreas = age_patches.select("labels").selfMask().addBands(ee.Image.pixelArea()).reduceConnectedComponents(ee.Reducer.sum(), 'labels', 256)
    # Select only patches of at least one hectare
    largePatches = patchAreas.gte(10000).selfMask()
    # ----------------  Exclude edge pixels ----------------
    # convert non-forest pixels from NA to zero
    # check what is the most frequent value within each hectare - if it's zero, it means the pixel is surrounded by non-forest cover
    exclude_edge_mask = largePatches.unmask(0).focalMode(kernelType = "circle", radius = 100, units = "meters").gt(0).selfMask()

    distance_to_border_mask = ee.Image(f"{data_folder}/distance_to_border_mask")

    floodable_forests = (ee.Image("projects/mapbiomas-public/assets/brazil/lulc/collection9/mapbiomas_collection90_integration_v1")
            .select(f"classification_{last_year}").eq(6))

    age = age.addBands([age.pixelLonLat().float(), floodable_forests]).updateMask(ipcc_mask.And(exclude_edge_mask).And(distance_to_border_mask)).rename("age", "floodable", "lon", "lat")

    export_image(age, name, region = roi, scale = 30)


export_secondary(tmf, f"tmf_{last_year}")
export_secondary(mapbiomas, f"mapbiomas_{last_year}")

## Processing other response data

Including ESA CCI Biomass, GEDI L4A, GEDI L2A

In [42]:
# Biomass in Mg/ha for 2020 from ESA CCI Biomass. Standard deviation was not removed since it increases with biomass values, so we would systematically remove pixels with higher biomass values.

# Biomass in Mg/ha for the specified year from ESA CCI Biomass
ESA_CCI = (ee.ImageCollection("projects/sat-io/open-datasets/ESA/ESA_CCI_AGB")
            .filterDate(f'{last_year}-01-01', f'{last_year}-12-31').first()
            .select('AGB')
            .toInt16()
            .rename(f"ESA_CCI_{last_year}"))

export_image(ESA_CCI, f"ESA_CCI_{last_year}", region=roi)

98.95065848290943

In [46]:
def quality_mask(image):
    image = (image.updateMask(image.select('l4_quality_flag').eq(1))
                 .updateMask(image.select('degrade_flag').eq(0)))
    
    relative_se = image.select('agbd_se').divide(image.select('agbd'))
    
    return image.updateMask(relative_se.lte(0.5))

GEDI_L4A = (ee.ImageCollection('LARSE/GEDI/GEDI04_A_002_MONTHLY')
            .filterDate(f'{last_year}-01-01', f'{last_year}-12-31')
            .map(quality_mask)
            .select('agbd').mean()
            .toInt16()                            
            .rename(f'GEDI_L4A_{last_year}'))

export_image(GEDI_L4A, f"GEDI_L4A_{last_year}", region = roi, scale = 30)

In [ ]:
def quality_mask(image):
    # Apply quality mask for GEDI 2A
    image = (image.updateMask(image.select('quality_flag').eq(1))
                 .updateMask(image.select('degrade_flag').eq(0)))
    return image

# Load GEDI 2A Canopy Height dataset
GEDI_L2A = (ee.ImageCollection('LARSE/GEDI/GEDI02_A_002_MONTHLY')
            .filterDate(f'{year}-01-01', f'{year}-12-31')
            .map(quality_mask)
            .select(['rh98'])
            .mean().toInt16()
            .rename(f'GEDI_L2A_{last_year}'))

# Export the image
export_image(GEDI_L2A, f"GEDI_L2A_{last_year}", region=roi, scale=30)